# FunctionGemma CLI Tool Recommender

Fine-tune FunctionGemma (270M) to recommend CLI tools based on:
- User queries
- Operating system (macOS, Linux, Ubuntu, Windows, etc.)
- Shell type (bash, zsh, fish, etc.)
- User preferences

**Requirements:** Free Colab T4 GPU

## 1. Install Dependencies

In [ ]:
%%capture
!pip install unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install datasets trl

## 2. Load Model and Tokenizer

In [ ]:
from unsloth import FastLanguageModel
import torch

# Model configuration
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/functiongemma-270m-it",
    max_seq_length=4096,
    load_in_16bit=True,
    full_finetuning=False,
)

print(f"Model loaded: {model.config._name_or_path}")
print(f"Parameters: {model.num_parameters():,}")

## 3. Add LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("LoRA adapters applied")

## 4. Prepare Training Data

In [ ]:
import json

# Sample training data (CLI tool recommendations)
training_examples = [
    {
        "query": "find all python files in the current directory",
        "os": "darwin",
        "shell": "zsh",
        "thinking": "The user wants to find Python files. On macOS with zsh: find is installed by default (BSD version). fd is a modern alternative but not installed by default.",
        "response": {
            "primary_tools": [{"name": "find", "category": "file_management", "confidence": 1.0, "reason": "Standard POSIX tool installed by default on macOS"}],
            "alternative_tools": [{"name": "fd", "category": "file_management", "install_cmd": "brew install fd", "reason": "Modern find alternative", "improvements": ["faster", "simpler syntax"]}],
            "task_category": "file_management"
        }
    },
    {
        "query": "search for the word error in all log files",
        "os": "ubuntu",
        "shell": "bash",
        "thinking": "The user wants to search for text in log files. On Ubuntu: grep is installed by default (GNU grep). ripgrep would be faster but requires installation.",
        "response": {
            "primary_tools": [{"name": "grep", "category": "search", "confidence": 1.0, "reason": "Standard GNU tool installed by default"}],
            "alternative_tools": [{"name": "rg", "category": "search", "install_cmd": "apt install ripgrep", "reason": "Fastest grep alternative", "improvements": ["faster", "respects gitignore"]}],
            "task_category": "search"
        }
    },
    {
        "query": "view a JSON API response and extract specific fields",
        "os": "darwin",
        "shell": "zsh",
        "thinking": "The user needs to parse JSON. curl is on macOS by default for fetching. jq is needed for parsing but not installed by default.",
        "response": {
            "primary_tools": [{"name": "curl", "category": "network", "confidence": 1.0, "reason": "Standard HTTP client installed by default"}],
            "alternative_tools": [{"name": "jq", "category": "json_processing", "install_cmd": "brew install jq", "reason": "Powerful JSON processor", "improvements": ["query syntax", "filtering", "transformation"]}],
            "task_category": "json_processing"
        }
    },
    {
        "query": "show disk usage of my home directory",
        "os": "linux",
        "shell": "bash",
        "thinking": "User wants to see disk usage. du is standard POSIX tool. ncdu is a better interactive alternative.",
        "response": {
            "primary_tools": [{"name": "du", "category": "disk", "confidence": 1.0, "reason": "Standard POSIX tool for directory space usage"}],
            "alternative_tools": [{"name": "ncdu", "category": "disk", "install_cmd": "apt install ncdu", "reason": "Interactive disk usage analyzer", "improvements": ["interactive UI", "visual browsing"]}],
            "task_category": "disk"
        }
    },
    {
        "query": "monitor system processes and CPU usage",
        "os": "darwin",
        "shell": "fish",
        "thinking": "Process monitoring on macOS. top is always available. htop provides better UI but needs installation.",
        "response": {
            "primary_tools": [{"name": "top", "category": "process", "confidence": 1.0, "reason": "Built-in process monitor"}],
            "alternative_tools": [{"name": "htop", "category": "process", "install_cmd": "brew install htop", "reason": "Interactive process viewer", "improvements": ["colored output", "mouse support", "tree view"]}],
            "task_category": "process"
        }
    }
]

def format_example(example):
    """Format a single example for FunctionGemma."""
    system = "You are a CLI tool recommendation assistant. Given the user's query, OS, shell, and preferences, recommend the most appropriate CLI tools. Format your response as a function call to recommend_tools."

    user_msg = f"OS: {example['os']}, Shell: {example['shell']}\nQuery: {example['query']}"

    model_response = f"<think>\n{example['thinking']}\n</think>\n"
    model_response += f"<start_function_call>call:recommend_tools{json.dumps(example['response'], indent=2)}<end_function_call>"

    formatted = f"<start_of_turn>developer\n{system}<end_of_turn>\n"
    formatted += f"<start_of_turn>user\n{user_msg}<end_of_turn>\n"
    formatted += f"<start_of_turn>model\n{model_response}<end_of_turn>"

    return formatted

# Format all examples
formatted_data = [format_example(ex) for ex in training_examples]

print(f"Prepared {len(formatted_data)} training examples")
print("\nSample formatted example:")
print("=" * 50)
print(formatted_data[0][:500] + "...")

## 5. Create Dataset

In [ ]:
from datasets import Dataset

dataset = Dataset.from_dict({"text": formatted_data})
print(f"Dataset size: {len(dataset)}")

## 6. Configure Trainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=4096,
    args=TrainingArguments(
        output_dir="./output",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        report_to="none",
    ),
)

print("Trainer configured")

## 7. Train

In [ ]:
print("Starting training...")
trainer.train()
print("Training complete!")

## 8. Test Inference

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

def get_recommendation(query, os_type="darwin", shell="zsh"):
    """Get CLI tool recommendations."""
    system = "You are a CLI tool recommendation assistant. Given the user's query, OS, shell, and preferences, recommend the most appropriate CLI tools. Format your response as a function call to recommend_tools."

    prompt = f"<start_of_turn>developer\n{system}<end_of_turn>\n"
    prompt += f"<start_of_turn>user\nOS: {os_type}, Shell: {shell}\nQuery: {query}<end_of_turn>\n"
    prompt += "<start_of_turn>model\n"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=1.0,
        top_k=64,
        top_p=0.95,
        do_sample=True,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    # Extract model response
    model_response = response.split("<start_of_turn>model\n")[-1]
    if "<end_of_turn>" in model_response:
        model_response = model_response.split("<end_of_turn>")[0]

    return model_response

# Test queries
test_queries = [
    "find all javascript files",
    "compress a folder into a zip file",
    "view running processes",
]

for query in test_queries:
    print(f"\n{'='*50}")
    print(f"Query: {query}")
    print("="*50)
    result = get_recommendation(query)
    print(result)

## 9. Save Model

In [ ]:
# Save LoRA adapters
model.save_pretrained("cli-tool-recommender-lora")
tokenizer.save_pretrained("cli-tool-recommender-lora")

print("Model saved to cli-tool-recommender-lora/")

# Optional: Save merged 16-bit model
# model.save_pretrained_merged("cli-tool-recommender-merged", tokenizer, save_method="merged_16bit")

# Optional: Save as GGUF for llama.cpp
# model.save_pretrained_gguf("cli-tool-recommender-gguf", tokenizer, quantization_method="q4_k_m")

## 10. Download Model

Run this cell to download your trained model.

In [ ]:
!zip -r cli-tool-recommender-lora.zip cli-tool-recommender-lora/

from google.colab import files
files.download('cli-tool-recommender-lora.zip')